# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import numpy as np
import statistics as stats
import sklearn
import re
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)



%matplotlib inline




#### Open the ` Employee.csv` file and name your dataset `employee`.

In [70]:
df = pd.read_csv('Employee.csv')


In [71]:
df.head(9)

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [72]:
# Method 1 - Groupby

df.groupby(['Department'])['Salary'].agg('mean')



Department
HR       45.00
IT       48.75
Sales    55.00
Name: Salary, dtype: float64

In [73]:
# Method 2 - Pivot

df.pivot_table(index= ['Department'], values = ['Salary'], aggfunc = ['mean']).head(5)

,mean
,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [74]:
# Method 2
df.pivot_table(index= ['Department','Title'], values = ['Salary'], aggfunc = ['mean']).head()


mean
                     Salary
Department Title           
HR         VP          70.0
           analyst     32.5
IT         VP          70.0
           analyst     32.5
           associate   60.0

In [75]:
df.pivot_table(index= ['Department','Title'], values = ['Salary'], aggfunc = ['mean']).reset_index()


,Department,Title,mean
,,,Salary
0,HR,VP,70.0
1,HR,analyst,32.5
2,IT,VP,70.0
3,IT,analyst,32.5
4,IT,associate,60.0
5,Sales,associate,55.0


#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [76]:
df.pivot_table(index= ['Department','Title'], columns='Years', values = ['Salary'], aggfunc = ['mean'])


mean                              
                     Salary                              
Years                     1     2     3     4     7     8
Department Title                                         
HR         VP           NaN   NaN   NaN   NaN   NaN  70.0
           analyst      NaN  30.0   NaN  35.0   NaN   NaN
IT         VP           NaN   NaN   NaN   NaN   NaN  70.0
           analyst     35.0  30.0   NaN   NaN   NaN   NaN
           associate    NaN   NaN   NaN   NaN  60.0   NaN
Sales      associate    NaN  55.0  55.0   NaN   NaN   NaN

In [77]:
df.pivot_table(index= ['Title','Years'], values = ['Salary'], aggfunc = ['mean'])



mean
                Salary
Title     Years       
VP        8         70
analyst   1         35
          2         30
          4         35
associate 2         55
          3         55
          7         60

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [78]:

df.pivot_table(index= ['Department', 'Title'], values = ['Salary','Name'], aggfunc = {'Salary':'mean','Name':'count'}).reset_index()

,Department,Title,Name,Salary
0,HR,VP,1,70.0
1,HR,analyst,2,32.5
2,IT,VP,1,70.0
3,IT,analyst,2,32.5
4,IT,associate,1,60.0
5,Sales,associate,2,55.0


#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [79]:
df_stats = df.pivot_table(index= ['Department','Title'], values = ['Salary'], aggfunc = ['mean','median',min,max,'std'])
df_stats

mean median    min    max       std
                     Salary Salary Salary Salary    Salary
Department Title                                          
HR         VP          70.0   70.0     70     70       NaN
           analyst     32.5   32.5     30     35  3.535534
IT         VP          70.0   70.0     70     70       NaN
           analyst     32.5   32.5     30     35  3.535534
           associate   60.0   60.0     60     60       NaN
Sales      associate   55.0   55.0     55     55  0.000000

In [80]:
df.describe()

,Years,Salary
count,9.000000,9.000000
mean,4.111111,48.888889
std,2.803767,16.541194
min,1.000000,30.000000
25%,2.000000,35.000000
50%,3.000000,55.000000
75%,7.000000,60.000000
max,8.000000,70.000000


 - Average salary for associate and VP titles is greater than the total salary average (48.9)
 - the analyst is the job title wth the lowest average salary
 - Mean and Median are equal therefore we can assume that there is no salary outliers
 - The department does not have any influence on the salary
 - Standard deviation is NaN when there is only one employee

#### Based on your comments, fill the missing values with an appropriate value.

In [81]:
df_stats['std'] = df_stats['std'].fillna(0)
df_stats

mean median    min    max       std
                     Salary Salary Salary Salary    Salary
Department Title                                          
HR         VP          70.0   70.0     70     70  0.000000
           analyst     32.5   32.5     30     35  3.535534
IT         VP          70.0   70.0     70     70  0.000000
           analyst     32.5   32.5     30     35  3.535534
           associate   60.0   60.0     60     60  0.000000
Sales      associate   55.0   55.0     55     55  0.000000

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [82]:
df_stakeholder = df.pivot_table(index= ['Department'], values = ['Salary','Name'], aggfunc = {'Salary':'sum','Name':'count'}).reset_index()
df_stakeholder = df_stakeholder.rename(columns={'Name' : 'number of employees'})
df_stakeholder

,Department,number of employees,Salary
0,HR,3,135
1,IT,4,195
2,Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [83]:
df.pivot_table(index= ['Department','Title'], values = ['Salary', 'Years'], aggfunc = {'Salary':'mean','Years':'mean'}).reset_index()
# Assumed that total years of work of a deparment is equal to the average of years worked by all employees of that department 

,Department,Title,Salary,Years
0,HR,VP,70.0,8.0
1,HR,analyst,32.5,3.0
2,IT,VP,70.0,8.0
3,IT,analyst,32.5,1.5
4,IT,associate,60.0,7.0
5,Sales,associate,55.0,2.5


### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [84]:
# your answer here
